In [1]:
import sys 
sys.path.append("../")

%cd ..

/home/manuel.couto.pintos/Documents/code/code_from_eRiskTrekitize/trekitize_fastApi


In [3]:
import os

os.environ['DB_MONGO'] = 'gambling2024_T3'

In [4]:
os.environ['DB_MONGO']

'gambling2024_T3'

In [2]:
%ls notebooks/data

BDI21.pdf  new_data/  qd_db/  runs/  runs_gambling/


# Preparación de los pools

## Primero leemos los relevantes de los participantes y nos quedamos con el top k de cada sistema

In [7]:
import pandas as pd
import os
from os import walk

k = 50

def insert_pools(folder='../runs', sep="[\s+,]", names=["query", "q0", "docid", "rank", "score", "system"]):
    pools = dict()
    for (root, dirs, files) in walk(folder):
        for name in files:
            
            path = os.path.join(root, name)
            df = pd.read_csv(path, sep=sep, engine='python', names=names)
            
            for query, q_df in df.groupby('query'):   
                q_df.loc[:,'rank'] = q_df['rank'].astype(pd.Int64Dtype())
                aux_set = pools.get(query, set())
                aux_list = q_df.sort_values(by='rank').reset_index(drop=True).head(k)['docid'].tolist()
                aux_set.update(set(aux_list))
                pools[query] = aux_set
    return pools
    
pools = insert_pools('notebooks/data/runs_gambling', sep="\t", names=["query", "rank", "docid", "text"])

In [8]:
from itertools import chain
rel_docs = list(set(chain.from_iterable(list(pools.values()))))
rel_docs

[43012,
 149511,
 51207,
 65545,
 225290,
 206860,
 344077,
 43024,
 49170,
 106516,
 266261,
 233493,
 256025,
 71706,
 96283,
 92187,
 290854,
 354344,
 335913,
 81969,
 14387,
 149555,
 65588,
 354361,
 354366,
 354367,
 51265,
 153668,
 84036,
 57426,
 69720,
 342107,
 266339,
 321637,
 215142,
 321641,
 247914,
 215150,
 354415,
 292976,
 325743,
 118901,
 77942,
 174198,
 176248,
 239736,
 311416,
 342138,
 104567,
 321663,
 188546,
 67715,
 227460,
 204934,
 168071,
 86158,
 266383,
 123025,
 270487,
 342167,
 161948,
 104608,
 223396,
 8357,
 274603,
 161964,
 325804,
 73902,
 65708,
 217262,
 65716,
 190645,
 55480,
 14524,
 63680,
 47297,
 43204,
 164040,
 110799,
 180432,
 313556,
 254168,
 88287,
 180449,
 243942,
 344297,
 295145,
 90353,
 39157,
 344310,
 110841,
 264442,
 18683,
 65791,
 45312,
 280831,
 47364,
 344327,
 102664,
 323850,
 241938,
 20755,
 63763,
 168218,
 297244,
 231710,
 248095,
 108835,
 264491,
 344368,
 332081,
 321843,
 71995,
 211260,
 178499,
 44

In [31]:
len(rel_docs)

17186

In [9]:
from os import listdir
from os.path import isfile, join
mypath = "notebooks/data/new_data/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

### Erisk 2023 depression

In [33]:
import xml.etree.ElementTree as ET

collection = {}

for file in onlyfiles:
    file_path = mypath + file
    file = open(file_path, 'r')
    tree = ET.fromstring(f'<DOCS>\n{file.read()}</DOCS>')
    for doc in tree:
        doc_no = doc.find('DOCNO').text
        text = doc.find('TEXT').text
        collection[doc_no] = text

### Erisk 2024 Gambling

In [11]:
collection = {}
all_df = []
for (root, dirs, files) in walk('notebooks/data/runs_gambling'):
    for name in files:
        path = os.path.join(root, name)
        df = pd.read_csv(path, sep="\t", engine='python', names=["query", "rank", "docid", "text"])
        all_df.append(df)
        
        
all_df = pd.concat(all_df)
        

In [14]:
print(len(all_df.index))
print(len(all_df.drop_duplicates().index))
all_df.drop_duplicates(inplace=True)

720
697


In [15]:
collection = dict(zip(all_df['docid'], all_df['text']))

## Una vez tenemos todo el contenido de los pools creamos una collección en qdrant que contenga únicamente los documentos que se encuentran en los pool

In [8]:
from qdrant_client.http.models import PointStruct

points = []
for _idx, doc_id in enumerate(rel_docs):
    
    points.append(
        PointStruct(
            id=_idx,
            vector={},
            payload={
                'doc_id': doc_id,
                'text': collection[doc_id]
            }
        )
    )


In [9]:
from qdrant_client import AsyncQdrantClient
from qdrant_client.http import models

client = AsyncQdrantClient("172.16.240.86", port=6333)

In [10]:
await client.create_collection(
    collection_name="treketize_docs_1",
    vectors_config={},
    on_disk_payload=True,
)

True

In [11]:
await client.upsert(
    collection_name="treketize_docs_1",
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [12]:
l = await client.retrieve(
    collection_name="treketize_docs_1",
    ids=[0, 3, 100],
)
l

[Record(id=100, payload={'doc_id': 's_1517_369_1', 'text': 'My weight loss secret is really dumb.'}, vector=None, shard_key=None),
 Record(id=0, payload={'doc_id': 's_975_69_6', 'text': "I can't make decisions on my own without having an anxiety attack because of my mom and I'll always resent her for it."}, vector=None, shard_key=None),
 Record(id=3, payload={'doc_id': 's_2163_509_6', 'text': 'But I believe that, as a species, human beings define their reality through suffering and misery.'}, vector=None, shard_key=None)]

## Scrapeamos las preguntas del bdi e insertamos el contenido en la base de datos

### Erisk 2023 depression

In [34]:
import PyPDF2 
import re
import pandas as pd
from trekitize_fastApi.model.query_model import QueryModel, QueryDao

# creating a pdf file object 

def parse_bdi(path='../BDI21.pdf'):
    pdfFileObj = open(path, 'rb') 
        
    # creating a pdf reader object 
    pdfReader = PyPDF2.PdfReader(pdfFileObj) 


    text1 = pdfReader.pages[0].extract_text()
    text2 = pdfReader.pages[1].extract_text()
    text3 = pdfReader.pages[2].extract_text()


    bdi_queries = []
    pre_num = False

    all_sections = []
    all_queries = []
    for text in [text1, text2, text3]:
        for section in re.findall(f'[0-9]+[\.a-b]([\w \'\-]+[^0-9])0', text):
            all_sections.append(section.strip())
        for idx,querie in enumerate(re.findall(f'([0-9]+)[\.a-b] ([\w \'\-,]+[^0-9])(?<![\.  ])',text)):
            all_queries.append(re.sub(r'\s{2,}',' ', querie[1].strip()))


    final = []    
    text = all_queries[0] + ":\n"
    for querie in all_queries[1:]:
        if querie not in all_sections:
            text = text + "\t" + querie + "\n"
        else: 
            final.append(text)
            text = querie + ":\n"
    final.append(text)

    QueryDao.drop()
    for q_num, text in enumerate(final):
        doc = QueryModel(query_num=(q_num+1), query_str=text)
        QueryDao.create(doc)

In [35]:
parse_bdi('notebooks/data/BDI21.pdf')

### eRisk 2024 gambling

In [32]:
import json
from trekitize_fastApi.model.query_model import QueryModel, QueryDao
with open('notebooks/data/queries.jsonl', "r") as f:
    QueryDao.drop()
    for line in f.readlines():
        query = json.loads(line)
        doc = QueryModel(query_num=int(query["_id"]), query_str=str(query['text']))
        QueryDao.create(doc)
    

### eRisk 2023 Depression y eRisk 2024 gambling

In [34]:
from trekitize_fastApi.model.pools_model import QueryPoolModel, QueryPoolDao, DocModel
from trekitize_fastApi.model.query_model import QueryModel, QueryDao

from rich import print

doc_map = dict(map(lambda x: (x[1],x[0]), enumerate(rel_docs)))

QueryPoolDao.drop()

for query_n, pool in pools.items():
    queryModel = QueryModel(**QueryDao.find_query_by_query_num(int(query_n)))
    poolModel = QueryPoolModel(
        query_id=queryModel.id,
        pool_list=list(map(lambda x: DocModel(doc_id=str(x), text=collection[x]), pool))
    )
    QueryPoolDao.create(poolModel)


## Una vez creados los pools hay que crear los juicios de los Usuarios

## Esta función es la encargada de generar los juicios a la hora de crear un usuario

In [3]:
from trekitize_fastApi.model.user_model import UserDao, UserModel
from trekitize_fastApi.model.pools_model import QueryPoolDao, QueryPoolModel
from trekitize_fastApi.model.user_pool import QueryJuiciosModel, UserPoolModel, UserPoolDao
from trekitize_fastApi.model.query_model import QueryDao, QueryModel


def add_user_pools(username: str):
    user = UserModel(**UserDao.find_by_username(username))
    pools = list(QueryPoolDao.findAll())
    
    user_pool = UserPoolDao.find_by_userid(user.id)
    if user_pool:
       UserPoolDao.delete(user_pool["_id"])
    
    userPool = UserPoolModel(user=user.id, juicios=list(map(lambda x: QueryJuiciosModel(), range(len(pools)))))
    
    print(UserPoolDao.create(userPool))

InsertOneResult(ObjectId('65cb96574490cf183a65717c'), acknowledged=True)
